### В данном ноутбуке проводится построение рекомендательной системы на основе пакета Surprise. Файл с рейтингами "Video Games" скачан с https://nijianmo.github.io/amazon/index.html

In [197]:
#импорт начальных библиотек
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

#assert python version >=3,5
import sys
assert sys.version_info >= (3,5)

import warnings

%matplotlib inline
display(HTML("<style>.container { width:80% !important; }</style>"))

warnings.filterwarnings("ignore")

Считаем файл с данными. Файл с рейтингами "VideoGames" был скачан с https://nijianmo.github.io/amazon/index.html 

In [35]:
df = pd.read_csv('data/Video_Games.csv', header=None)
df.columns = ["item","user","rating","timestamp"]

In [36]:
df.head()

,item,user,rating,timestamp
0,0439381673,A21ROB4YDOZA5P,1.0,1402272000
1,0439381673,A3TNZ2Q5E7HTHD,3.0,1399680000
2,0439381673,A1OKRM3QFEATQO,4.0,1391731200
3,0439381673,A2XO1JFCNEYV3T,1.0,1391731200
4,0439381673,A19WLPIRHD15TH,4.0,1389830400


**Проведем базовый EDA**

In [179]:
print("Num of ratings {:d}, num of users {:d}, num of items {:d}".format(df.shape[0], df.user.nunique(), df.item.nunique()))

Num of ratings 2565349, num of users 1540618, num of items 71982


Распределение рейтигов:

In [41]:
df.rating.value_counts() / df.shape[0]

5.0    0.579791
4.0    0.160763
1.0    0.121578
3.0    0.082775
2.0    0.055093
Name: rating, dtype: float64

В основом, (57%), положительные рейтинги. Меньше всего рейтингов, с оценко 3 и 2, - 8.3% и 5,5% соответственно. Пропусков, т.е. 0 рейтингов в данных нет

In [181]:
print("% of absent data in user-item matrix {:0.6f}".format(1- df.shape[0] / (df.user.nunique()*df.item.nunique())))

% of absent data in user-item matrix 0.999977


В матрице Юзер-Айтем будет достаточно много пропусков, что негативно скажется на модели. Рекомендованное кол-во пропусков не более 99.5%

**Подготовим тренировачный и тестовый наборы**

В данных присутствует поле timestamp. Отсортируем данные по времени и возмем 30% из будущего как тестовый набор. При этом в тренировачном наборе занули рейтинги из тестового набора, чтобы потом оценить соответствие реальным рейтингам.

In [42]:
df.sort_values(by="timestamp", inplace=True)

In [51]:
#Shuffle = False, so there will be no radndom shuffling in data
df_train, df_test = train_test_split(df, shuffle=False, test_size=0.3, random_state=143)

In [52]:
#zero the ratings for users & items who present in test dataset
df_train_zero = df_train.copy()
df_train_zero.loc[df_train_zero[df_train_zero.set_index(['item','user']).index.isin(df_test.set_index(['item','user']).index)].index, 'rating'] = 0

In [183]:
df_train_zero[df_train_zero.rating == 0].shape[0]

54

Получилось всего 54 оценки по которым мы можем сравнить реально поставленные рейтинги из будущего

**Построим рекомендательную модель с использованием библиотеки Surprise**

In [81]:
from surprise import accuracy, NormalPredictor, Dataset, Reader
from surprise.model_selection import cross_validate

In [55]:
#read data from train dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_train_zero[["user", "item", "rating"]], reader)

Оценим бейз-лайн метрики с помощью базового рекомендатора NormalPredictor , который просто строит прогноз исходя из предположения о нормальности распределения рейтингов для каэдого юзерп

In [184]:
cross_validate(NormalPredictor(), data, cv=3)

{'test_rmse': array([1.76837981, 1.76418789, 1.76670339]),
 'test_mae': array([1.35423059, 1.35066578, 1.35323318]),
 'fit_time': (0.874598503112793, 1.0558373928070068, 1.0620834827423096),
 'test_time': (3.1861259937286377, 3.1254053115844727, 3.1237478256225586)}

Бейзлайн по RMSE составляет 1,766

Возмем в качестве рекомендатора алгоритм SVD и проведем его тренировку и поиск лучших параметров по сетке

In [57]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

In [58]:
param_grid = {"n_factors":[10,20,50,100,150], "n_epochs": [5, 10, 20], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

In [186]:
print("Best scores: RMSE {:0.4f} , MAE {:0.4f}".format(gs.best_score["rmse"], gs.best_score["mae"]))

Best scores: RMSE 1.2622 , MAE 1.0058


По сравнению с бейзлайном ошибка RMSE снизился на 0.521

In [61]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [187]:
#grid search iterations details
results_df.head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,...,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_n_epochs,param_lr_all,param_reg_all
0,1.305402,1.306766,1.300524,1.304231,0.002679,51,1.043563,1.043977,1.040365,1.042635,...,51,1.743582,0.039521,2.574891,0.026587,"{'n_factors': 10, 'n_epochs': 5, 'lr_all': 0.0...",10,5,0.002,0.4
1,1.309075,1.310370,1.304255,1.307900,0.002631,56,1.049138,1.049550,1.046002,1.048230,...,56,1.413548,0.027842,2.325187,0.338944,"{'n_factors': 10, 'n_epochs': 5, 'lr_all': 0.0...",10,5,0.002,0.6
2,1.288240,1.289552,1.283519,1.287103,0.002590,32,1.030407,1.030869,1.027597,1.029624,...,31,1.407524,0.033631,2.332910,0.354610,"{'n_factors': 10, 'n_epochs': 5, 'lr_all': 0.0...",10,5,0.005,0.4
3,1.292757,1.294058,1.288185,1.291667,0.002519,41,1.037404,1.037914,1.034636,1.036651,...,41,1.376340,0.040936,2.505179,0.015699,"{'n_factors': 10, 'n_epochs': 5, 'lr_all': 0.0...",10,5,0.005,0.6
4,1.292457,1.293848,1.287641,1.291315,0.002660,36,1.032996,1.033438,1.029902,1.032112,...,36,2.644036,0.120965,2.281314,0.354578,"{'n_factors': 10, 'n_epochs': 10, 'lr_all': 0....",10,10,0.002,0.4


Лучшие параметры:

In [64]:
print(gs.best_params["rmse"])

{'n_factors': 150, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


Теперь обучим модель на всем тренировачном сете и оценим на тестовом

In [69]:
algo_svd = gs.best_estimator["rmse"]
algo_svd.fit(data.build_full_trainset())

In [79]:
data_test = Dataset.load_from_df(df_test[["user", "item", "rating"]], reader)
test_set = data_test.build_full_trainset().build_testset()
predictions = algo_svd.test(test_set)

In [188]:
# Then compute RMSE & MAE on test dataset
print("Metrics on test data: RMSE {:0.4f}, MAE {:0.4f}".format(accuracy.rmse(predictions), accuracy.mae(predictions)))

RMSE: 1.4214
MAE:  1.1451
Metrics on test data: RMSE 1.4214, MAE 1.1451


In [193]:
pd.DataFrame([round(p.est) for p in predictions], columns=["rating"]).value_counts()

rating
4         711281
3          31252
5          26750
2            314
1              8
dtype: int64

На тестовом сете ошибки выросли, RMSE на 0.23 MAE на 0.135, получается достаточно смещенная оценка в сторону средних значений рейтингов

Оценим теперь качетсво на той выборке, по которой мы зануляли рейтинги

In [163]:
actuals = []
estimates = []
for item, user, rating,_ in df_train_zero[df_train_zero.rating == 0].itertuples(index=False):
    actual_rating = df_test[(df_test.item == item) & (df_test.user == user)]["rating"].values[0]
    actuals.append(actual_rating)
    est = algo_svd.predict(user, item, r_ui=actual_rating).est
    estimates.append(est)

In [172]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [194]:
print("Estimates on users and items who have ratings in future:  RMSE {:0.4f}, MAE {:0.4f}, MAE% {:0.2f}".format(
    mean_squared_error(actuals, estimates), 
    mean_absolute_error(actuals, estimates),
    mean_absolute_percentage_error(actuals, estimates)
))

Estimates on users and items who have ratings in future:  RMSE 1.9697, MAE 1.2152, MAE% 0.53


In [196]:
pd.DataFrame([round(e) for e in estimates], columns=["rating"]).value_counts()

rating
4         40
3          9
5          4
2          1
dtype: int64

**На контрольной выборке ошибки достаточно велики. Видимо из-за слишком большого кол-ва пропусков в исходном наборе. Также можно попробовать другие алгоритмы из библиотеки.**